In [1]:
import gzip

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [2]:
from collections import defaultdict
import numpy as np
data = []
count = 0;

for l in readGz("train.json.gz"):
    data.append(l)

trainData, validData = np.array_split(data, 2)

print(len(trainData))
print(len(validData))
print(trainData[0])

100000
100000
{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}


In [3]:
businessCount = defaultdict(int)
visitedBusinesses = defaultdict(int)
users = defaultdict(int)
totalVisits = 0;

for d in trainData:
    user, business = d['userID'], d['businessID']
    businessCount[business] += 1
    totalVisits += 1
    visitedBusinesses[user+'-'+business] += 1
    users[user] += 1

print(len(businessCount))
print(len(users))
print(len(visitedBusinesses))

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalVisits/2: break

20490
18052
100000


In [4]:
businessList = []
usersList = []

'''for c in businessCount:
    businessList.append(c)
for u in users:
    usersList.append(u)''' #Not taking the pairs from training data, taking them from validation data

vbusinessCount = defaultdict(int)
vvisitedBusinesses = defaultdict(int)
vusers = defaultdict(int)
vtotalVisits = 0;
vnotvisited = defaultdict(int)

for d in validData:
    user, business = d['userID'], d['businessID']
    vbusinessCount[business] += 1
    vtotalVisits += 1
    vvisitedBusinesses[user+'-'+business] += 1
    vusers[user] += 1
    
print(len(vvisitedBusinesses))
print(len(vbusinessCount))
print(len(vusers))

for c in vbusinessCount:
    businessList.append(c)
for u in vusers:
    usersList.append(u)

import random

while(len(vnotvisited)<100000):
    pair = random.choice(usersList)+'-'+random.choice(businessList)
    while(pair in vvisitedBusinesses):
        pair = random.choice(usersList)+'-'+random.choice(businessList)
    vnotvisited[pair] += 1

print(len(vnotvisited))

100000
20521
18079
100000


In [5]:
wrongPred = 0;
correctPred = 0;

def baselinePredict(userBusinessPair, visited):
    global wrongPred
    global correctPred
    global return1
    print(wrongPred)
    for p in userBusinessPair:
        u,b = p.strip().split('-')
        if((b in return1 and visited) or (b not in return1 and not visited)):
            correctPred += 1
        else:
            wrongPred += 1

baselinePredict(vvisitedBusinesses, True)
baselinePredict(vnotvisited, False)

#print(wrongPred)
#print(correctPred)
print(wrongPred/(wrongPred+correctPred)*100)

0
55784
37.6445


In [6]:
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.65*totalVisits: break

wrongPred = 0
correctPred = 0
baselinePredict(vvisitedBusinesses, True)
baselinePredict(vnotvisited, False)

print(wrongPred/(wrongPred+correctPred)*100)

0
43479
37.835


In [24]:
visitedCategory = defaultdict(set)
print(trainData[0])

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}


In [25]:
for d in trainData:
    for c in d['categories']:
        visitedCategory[d['userID']].add(c)
        
print("done")
print(len(visitedCategory))

done
18052


In [26]:
bCategory = defaultdict(set)
print(validData[5])

{'rating': 4.0, 'businessID': 'B903181568', 'reviewText': "This is a great spot.  You could eat really cheaply or more... Good local hangout, and the bartenders are great.  Good mac and cheese....good salads, good 'bad food', and great 'quarter pounder' cookies!", 'userID': 'U455837535', 'reviewTime': 'May 3, 2013', 'categories': ['American Restaurant'], 'reviewHash': 'R506914583', 'unixReviewTime': 1367595704, 'categoryID': 0}


In [27]:
for d in validData:
    for c in d['categories']:
        bCategory[d['businessID']].add(c)
print("done")
print(len(bCategory))

done
20521


In [28]:
wPred = 0
cPred = 0

def categoryVisitedPred(ubPairs, visited):
    global wPred
    global cPred
    for p in ubPairs:
        u,b = p.strip().split('-')
        match = False;
        for c in bCategory[b]:
            for uc in visitedCategory[u]:
                if(c==uc):
                    match = True
                    break
            if(match):
                break
        if((match and visited) or (not match and not visited)):
            cPred += 1
        else:
            wPred += 1

categoryVisitedPred(vvisitedBusinesses, True)
categoryVisitedPred(vnotvisited, False)

print(wPred)
print(cPred)
print(wPred/(wPred+cPred))

69473
130527
0.347365


In [22]:
visitedCategory = defaultdict(set)
bCategory = defaultdict(set)

for d in data:
    #print(d)
    for c in d['categories']:
        visitedCategory[d['userID']].add(c)
        bCategory[d['businessID']].add(c)
print("done")
print(len(visitedCategory))
print(len(bCategory))

done
18793
21321


In [23]:
def categoryVisitedPredFile(inputFile, outputFile):
    predictions = open(outputFile, 'w')
    for l in open(inputFile):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u,b = l.strip().split('-')
        match = False;
        for c in bCategory[b]:
            for uc in visitedCategory[u]:
                if(c==uc):
                    print(c)
                    match = True
                    break
            if(match):
                break
        if(match):
            predictions.write(u + '-' + b + ",1\n")
        else:
            predictions.write(u + '-' + b + ",0\n")

categoryVisitedPredFile ("pairs_Visit.txt", "predictions_Visit.txt")    

print("done")

Mediterranean Restaurant
Latin American Restaurant
Frozen Yogurt Shop
Furniture Store
Latin American Restaurant
Eclectic Restaurant
Seafood Restaurant
Burmese Restaurant
Hamburger Restaurant
Latin American Restaurant
American Restaurant
Cafe
European Restaurant
Dessert Shop
Pizza Restaurant
Day Spa
Coffee Shop
Latin American Restaurant
American Restaurant
Latin American Restaurant
American Restaurant
Bar
Bar
Restaurant
Bar
Pizza Restaurant
American Restaurant
Southern Restaurant (US)
American Restaurant
Latin American Restaurant
Ice Cream Shop
Cocktail Bar
Bar
Coffee Shop
Pizza Restaurant
Dessert Shop
Korean Restaurant
American Restaurant
European Restaurant
Chinese Restaurant
Hamburger Restaurant
Bar
Movie Theater
Non-Profit Organization
Bakery
Hamburger Restaurant
Motel
Hotel
American Restaurant
Bar
American Restaurant
Brewery
Asian Restaurant
American Restaurant
Tea House
Cafe
American Restaurant
Bakery
American Restaurant
Asian Restaurant
American Restaurant
American Restaurant
Bar

Performing Arts Theater
Hamburger Restaurant
Barbecue Restaurant
Coffee Shop
Latin American Restaurant
Restaurant
Seafood Restaurant
Caterer
Fast Food Restaurant
American Restaurant
Italian Restaurant
Live Music Venue
South American Restaurant
American Restaurant
Lounge
Bar & Grill
American Restaurant
Kosher Restaurant
Latin American Restaurant
American Restaurant
American Restaurant
European Restaurant
American Restaurant
Steak House
Restaurant
American Restaurant
Italian Restaurant
Sports Bar
Bakery
Seafood Restaurant
Asian Restaurant
Bar
Clothing Store
American Restaurant
Bar & Grill
Used Car Dealer
Department Store
Toy Store
Seafood Restaurant
Steak House
Latin American Restaurant
Pediatric Dentist
European Restaurant
American Restaurant
Asian Restaurant
Coffee Shop
Steak House
American Restaurant
European Restaurant
Middle Eastern Restaurant
Mediterranean Restaurant
Cafe
Mediterranean Restaurant
Cocktail Bar
Pizza Restaurant
Electronics Store
Seafood Restaurant
Coffee Shop
Italian

Coffee Shop
Italian Restaurant
Hamburger Restaurant
Latin American Restaurant
Hotel
Used Car Dealer
Restaurant
Hamburger Restaurant
Seafood Restaurant
American Restaurant
Fast Food Restaurant
Lounge
Lobster Restaurant
Latin American Restaurant
American Restaurant
Asian Restaurant
Movie Theater
Bakery
Cafe
Used Car Dealer
European Restaurant
Asian Restaurant
Restaurant
Asian Restaurant
European Restaurant
Latin American Restaurant
Cafe
Coffee Shop
French Restaurant
American Restaurant
Department Store
Bar
Asian Restaurant
Seafood Restaurant
Lounge
Bar
American Restaurant
Italian Restaurant
Latin American Restaurant
Bar
American Restaurant
Coffee Shop
Bar
American Restaurant
Italian Restaurant
Fast Food Restaurant
Asian Restaurant
Californian Restaurant
Asian Restaurant
Brazilian Restaurant
Lobster Restaurant
Bar & Grill
Park
Barbecue Restaurant
American Restaurant
Animal Hospital
French Restaurant
Asian Restaurant
American Restaurant
American Restaurant
Italian Restaurant
Pharmacy
Seafo

Used Car Dealer
Mexican Restaurant
Hamburger Restaurant
Lounge
Bar
Cafe
American Restaurant
Italian Restaurant
Seafood Restaurant
American Restaurant
American Restaurant
Bar
Italian Restaurant
Japanese Restaurant
Creole Restaurant
Hamburger Restaurant
American Restaurant
Sports Bar
Bar
Steak House
American Restaurant
Bar
American Restaurant
Asian Restaurant
Brunch Restaurant
American Restaurant
Latin American Restaurant
Bar
Italian Restaurant
American Restaurant
Hamburger Restaurant
Latin American Restaurant
Latin American Restaurant
Asian Restaurant
Coffee Shop
Bar
Dessert Restaurant
Asian Restaurant
American Restaurant
Dessert Shop
Cocktail Bar
Restaurant
Hamburger Restaurant
Property Management Company
Lounge
Sushi Restaurant
Bar
Fine Dining Restaurant
Coffee Shop
Fast Food Restaurant
Steak House
Seafood Restaurant
Pizza Restaurant
Coffee Shop
Asian Restaurant
Latin American Restaurant
Bar
Italian Restaurant
American Restaurant
Bar
Vegan Restaurant
Asian Restaurant
Seafood Restauran

Performing Arts Theater
Mexican Restaurant
Lounge
Grocery Store
Fast Food Restaurant
Window Treatment Store
Mexican Restaurant
Italian Restaurant
Vegetarian Restaurant
Bar
French Restaurant
Bar
Bakery
American Restaurant
European Restaurant
Pizza Restaurant
Auto Repair Shop
Cafe
American Restaurant
Italian Restaurant
Asian Restaurant
Italian Restaurant
Hamburger Restaurant
Cafe
Dessert Restaurant
Latin American Restaurant
Coffee Shop
Cafe
Californian Restaurant
Italian Restaurant
Asian Restaurant
Italian Restaurant
Italian Restaurant
Latin American Restaurant
Diner
Bar
Coffee Shop
Italian Restaurant
Shopping Mall
Brewpub
Hamburger Restaurant
American Restaurant
American Restaurant
Night Club
Dessert Shop
Diner
European Restaurant
Stadium
Latin American Restaurant
Cafe
Mexican Restaurant
Fast Food Restaurant
Book Store
Pizza Restaurant
Asian Restaurant
Bar & Grill
American Restaurant
Fast Food Restaurant
American Restaurant
American Restaurant
Asian Restaurant
Sports Bar
American Restau

...

Restaurant
Bar
Fast Food Restaurant
Botanical Garden
International Airport
Concert Hall
Lounge
Bar
Mexican Restaurant
Mediterranean Restaurant
Sandwich Shop
Italian Restaurant
Latin American Restaurant
Asian Restaurant
Movie Theater
Live Music Venue
Asian Restaurant
Latin American Restaurant
European Restaurant
Latin American Restaurant
Asian Restaurant
Cafe
Kennel
Cafe
Used Truck Dealer
Asian Restaurant
Steak House
Car Service
Hamburger Restaurant
Pizza Restaurant
Hamburger Restaurant
Brewery
Asian Restaurant
Used Car Dealer
Restaurant
Bar
Asian Restaurant
European Restaurant
Pizza Restaurant
American Restaurant
Coffee Shop
Lounge
European Restaurant
Coffee Shop
American Restaurant
American Restaurant
Used Car Dealer
European Restaurant
Italian Restaurant
American Restaurant
American Restaurant
Diner
Bar
Bar & Grill
American Restaurant
American Restaurant
European Restaurant
Restaurant
Live Music Venue
Steak House
Asian Restaurant
American Restaurant
Ice Cream Shop
Italian Restaurant


Sports Bar
Latin American Restaurant
Fast Food Restaurant
American Restaurant
American Restaurant
American Restaurant
Italian Restaurant
Movie Theater
Asian Restaurant
American Restaurant
Seafood Restaurant
Asian Restaurant
Bar
Used Car Dealer
Cafe
Restaurant
Bar
Musical Instrument Store
Coffee Shop
Cafe
Latin American Restaurant
American Restaurant
Hamburger Restaurant
Cafe
Seafood Restaurant
Asian Restaurant
Mexican Restaurant
Toy Store
Coffee Shop
Asian Restaurant
Night Club
Seafood Restaurant
Asian Restaurant
Asian Restaurant
Restaurant
Cafe
French Restaurant
Latin American Restaurant
Performing Arts Theater
Japanese Restaurant
Italian Restaurant
Restaurant
Italian Restaurant
Asian Restaurant
American Restaurant
Cafe
Bar
Italian Restaurant
American Restaurant
Fast Food Restaurant
Latin American Restaurant
American Restaurant
European Restaurant
American Restaurant
Italian Restaurant
Bar & Grill
Diner
Italian Restaurant
Ice Cream Shop
American Restaurant
Resort
Used Car Dealer
Asian

American Restaurant
Sandwich Shop
Coffee Shop
Day Spa
Diner
American Restaurant
Pub
Lounge
Restaurant
Gift Shop
Bar & Grill
Mexican Restaurant
Ice Cream Shop
Bistro
Grocery Store
Hamburger Restaurant
Montessori School
Brewpub
Bar
Pawn Shop
American Restaurant
Sandwich Shop
American Restaurant
Asian Restaurant
Italian Restaurant
Asian Restaurant
Public Swimming Pool
Italian Restaurant
Asian Restaurant
French Restaurant
Asian Restaurant
Wine Bar
Live Music Venue
French Restaurant
Sushi Restaurant
Hamburger Restaurant
Hamburger Restaurant
Fast Food Restaurant
Liquor Store
Italian Restaurant
American Restaurant
Vegetarian Restaurant
Auto Repair Shop
Middle Eastern Restaurant
Latin American Restaurant
American Restaurant
Asian Restaurant
Clothing Store
Steak House
Latin American Restaurant
Italian Restaurant
American Restaurant
Thai Restaurant
Dental Clinic
American Restaurant
Italian Restaurant
Latin American Restaurant
Pizza Restaurant
French Restaurant
Pizza Restaurant
Lounge
Diner
Baker

American Restaurant
French Restaurant
American Restaurant
Vegan Restaurant
Bakery
Steak House
Asian Restaurant
Middle Eastern Restaurant
Italian Restaurant
Restaurant
Vegetarian Restaurant
Asian Restaurant
Bistro
Asian Restaurant
Italian Restaurant
American Restaurant
Asian Restaurant
Italian Restaurant
Coffee Shop
French Restaurant
Latin American Restaurant
Asian Restaurant
French Restaurant
Italian Restaurant
Asian Restaurant
Steak House
Used Car Dealer
Asian Restaurant
Bar
Latin American Restaurant
Restaurant
European Restaurant
Hamburger Restaurant
European Restaurant
Bar
Fast Food Restaurant
Italian Restaurant
Lounge
Book Store
Latin American Restaurant
Brewpub
Pizza Restaurant
Bar & Grill
Deli
Coffee Shop
Book Store
Asian Restaurant
Lounge
Caribbean Restaurant
Mexican Restaurant
Italian Restaurant
Coffee Shop
European Restaurant
Dessert Restaurant
Italian Restaurant
Department Store
Asian Restaurant
Bistro
American Restaurant
Chinese Restaurant
Vegetarian Restaurant
Latin America

Cafe
Bar
American Restaurant
Asian Restaurant
Used Car Dealer
Used Car Dealer
Bar
Bar
Chinese Restaurant
Asian Restaurant
American Restaurant
Cafe
Mexican Restaurant
American Restaurant
Italian Restaurant
Bar
Dessert Shop
Italian Restaurant
Bar
American Restaurant
Bar
Southeast Asian Restaurant
Filipino Restaurant
Bar & Grill
Buffet Restaurant
Coffee Shop
Adult Entertainment Club
Used Car Dealer
Used Car Dealer
Bar
Seafood Restaurant
Southern Restaurant (US)
Asian Restaurant
Coffee Shop
General Contractor
Cafe
Cafe
American Restaurant
Sports Bar
Bar
Asian Restaurant
Live Music Venue
Coffee Shop
Hotel
American Restaurant
Electronics Store
American Restaurant
Asian Restaurant
Italian Restaurant
Barbecue Restaurant
Fast Food Restaurant
Asian Restaurant
Sushi Restaurant
Chinese Restaurant
American Restaurant
Steak House
Deli
American Restaurant
Asian Restaurant
American Restaurant
Asian Restaurant
Coffee Shop
Bar & Grill
Luxury Hotel
Restaurant
Lounge
Asian Restaurant
Auto Repair Shop
Lati